In [1]:
import numpy as np
import pandas as pd
import pystan
import os
from codebase.file_utils import save_obj, load_obj
from codebase.model_fit_bin import get_lgscr
import argparse
from pdb import set_trace

In [2]:
log_dir = 'log/bin_sim_data/ppp/20201201_133143_rerunbin_s2m1/'
ps = load_obj('ps', log_dir)

In [3]:
def form_df(samples, num_chains, rows):
    dfs = []
    for cn in range(num_chains):
        for r in range(rows):
            if rows>1:
                df = pd.DataFrame(
                        samples[:,cn, r]
                )
            else:
                df = pd.DataFrame(
                        samples[:,cn]
                )
            df.insert(0, 'idx', np.arange(df.shape[0]))
            df = df.melt(id_vars ='idx', var_name = 'col')
            df.insert(1, 'row' , r)
            df.insert(1, 'chain', cn)
            dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def get_post_df(samples):
    num_chains = samples.shape[1]
    if samples.ndim > 3:
        rows = samples.shape[2]
        df = form_df(samples, num_chains, rows)
    else:
        rows = 1
        df = form_df(samples, num_chains, 1)
    return df

In [4]:
dfs=[]
for name in ps.keys():
    df = get_post_df(ps[name])
    df['name'] = name
    dfs.append(df)

In [7]:
ps.keys()

odict_keys(['beta', 'alpha', 'zz', 'Phi_cov', 'yy'])

In [5]:
outdf = pd.concat(dfs)

In [6]:
outdf.to_csv("log/test_kostas.csv", index=False)

SystemError: <built-in method item of numpy.ndarray object at 0x7f31b47fcdf0> returned a result with an error set